In [1]:
from google.cloud import bigquery
from google.oauth2 import service_account
key_path = "./creds/service_acc_auth.json"
credentials = service_account.Credentials.from_service_account_file(key_path, scopes = ["https://www.googleapis.com/auth/cloud-platform"])
bigquery_client = bigquery.Client(credentials=credentials, project=credentials.project_id)

In [2]:
# query new data
union_weeks = """
SELECT * , 
FROM `scanning-database.scanning_data_backup.W20` 
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W21`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W22` 
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W23` 
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W24` 
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W25` 
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W26` 
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W27` 
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W28` 
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W29`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W30`
"""

In [3]:
# Overwriting old PowerBI source data
job_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10,
destination = 'scanning-database.PowerBI.PowerBI',
write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE
) # set query limit to 10G
query_job = bigquery_client.query(union_weeks, job_config)  # API request
rows = query_job.result()  # Waits for query to finish
data_in_use = rows.to_dataframe()

In [4]:
data_in_use.head()

,Start_Time,Finish_Time,Activity,Seq_Code,Recipe_Name,Break_Reasons,Missing_Ingredients,Kitting_Line,Assembly_Batch,Event_Shift,Team_Leader,Pickers_Count,Time_Consumption,Week
0,2020-05-20 18:28:00+00:00,2020-05-20 18:28:00+00:00,Production,None,F2,None,None,KL1,None,Wednesday Afternoon,Kim,0,0.22,2020-22
1,2020-05-24 06:27:00+00:00,2020-05-24 06:27:00+00:00,Preparation/Changeover,None,B4,None,None,KL9,None,Sunday Morning,Dummy,0,0.18,2020-22
2,2020-06-05 13:30:00+00:00,2020-06-06 08:12:00+00:00,Factory Closed,None,H2,None,None,KL7,None,Saturday Morning,Test,0,1122.13,2020-24
3,2020-05-24 10:51:00+00:00,2020-05-24 10:51:00+00:00,Production,None,J4,None,None,KL15,None,Sunday Morning,Dummy,0,0.22,2020-22
4,2020-05-31 10:45:00+00:00,2020-05-31 11:40:00+00:00,Break,None,R4,move to other line,None,KL18,None,Sunday Morning,Dummy,0,55.58,2020-23


In [5]:
data_in_use.shape

(78137, 14)